In [1]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00


In [2]:
#packages used
from importlib.metadata import version
import tiktoken
import torch

print("torch version: ",version('torch'))
print("tiktoken version:", version('tiktoken'))

torch version:  2.3.0+cu121
tiktoken version: 0.7.0


In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
  url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
  file_path= "the-verdict.txt"
  urllib.request.urlretrieve(url,file_path)

In [4]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

print("total number of characters: ",len(raw_text))
print(raw_text[:99])

total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
#tokenize and embed this text for llm
#build tokenizer


In [6]:
#split based on space
import re
text = "hello, World. This, is a test."
result = re.split(r'(\s)',text)
print(result)

['hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [7]:
#split on commas and periods as well
result = re.split(r'([,.]|\s)',text)
print(result)

['hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
#remove empty spaces
result = [item for item in result if item.strip()]
print(result)

['hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


In [9]:
#handle other types of punctuation
text = "Hello world. Is this, a test?"
result = re.split(r'([,.!?:;_()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)

['Hello', 'world', '.', 'Is', 'this', ',', 'a', 'test', '?']


In [10]:
#can now apply to raw text
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [11]:
#calcualte total number of tokens
print(len(preprocessed))

4690


In [12]:
#converting tokens into token ids
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [13]:
vocab = {token: integer for integer, token in enumerate(all_words)}

In [14]:
for i, item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [15]:
#putting it all together in tokenizer class
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s, i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)',text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

    # Replace spaces before the specified punctuations


In [16]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)



[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [17]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [18]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [19]:
#adding special context tokens
#unknown, end of sentence

In [20]:
text = "Hello how are you?"
tokenizer.encode(text)
#error since hello not present in vocab

KeyError: 'Hello'

In [21]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])


vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [22]:
len(vocab.items())

1132

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [24]:
#adjust tokenizer to handle unk token

class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {v: i for i,v in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)

    preprocessed = [item.strip() for item in preprocessed if item.strip()]

    preprocessed = [item if item in self.str_to_int
                    else "<|unk|>" for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

In [25]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [26]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [27]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

Bytepair encoding

In [28]:
#breakdown of unknown words into smaller sub words
#to handle out of voc words
#use BPE tokenizer of openai from tiktoken


In [29]:
import importlib
import tiktoken

print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.7.0


In [30]:
tokenizer = tiktoken.get_encoding("gpt2")

In [31]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [32]:
strings= tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [33]:
#data sampling with sliding window
#predict one word at a time
#can not access words past the target


In [35]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [36]:
# For each text chunk, we want the inputs and targets
# Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [37]:
enc_sample = enc_text[50:]

In [40]:
context_size =4
x= enc_text[:context_size]
y=enc_text[1:context_size+1]

print(f"x:{x}")
print(f"y:    {y}")

x:[40, 367, 2885, 1464]
y:    [367, 2885, 1464, 1807]


In [41]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(context, "----->",desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [46]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(tokenizer.decode(context),"----->",tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [47]:
#input and targets that are shifted by one

In [48]:
import torch
print(torch.__version__)

2.3.0+cu121


In [ ]:
#dataset and dataloader that extracts chunks from